# Demo 1 - Data Prep

In order to run this notebook locally, you will need the following packages installed. All are installable via pip (`python3 -m pip install <package-name>`).
- numpy
- astropy
- h5py

In [8]:
import h5py, numpy as np, astropy.units as u, astropy.constants as constants
from vortrace import vortrace as vt
mol_frac = 4.0 / (1.0 + 3.0 * 0.76);

In [3]:
with h5py.File("../data/galaxy.hdf5",'r') as f:
    pos = f['PartType0/Coordinates'][:]
    pos -= f['PartType5/Coordinates'][0]
    mass = f['PartType0/Masses'][:]
    rho = f['PartType0/Density'][:]

rho = (np.array(rho).astype(np.float64)*1e10*u.Msun/u.kpc**3/(mol_frac*constants.m_p.to(u.Msun))).to(u.cm**-3).value

In [4]:
boxsz = 300
boundbox = np.array([[-boxsz,boxsz] for i in range(3)])
imsz = 100
nres = 512
center = [0,0,0]
bounds = [-imsz,imsz]
extent = [[-imsz,imsz],[-imsz,imsz]]

pc = vt.ProjectionCloud(pos,rho,boundbox=np.ravel(boundbox))
vmap1 = pc.grid_projection(extent,nres,bounds,center,proj='xy')*u.kpc.to(u.cm)
vmap2 = pc.grid_projection(extent,nres,bounds,center,proj='xz')*u.kpc.to(u.cm)

Applied bounding box filter: 2387832 -> 1869391 particles
Loading pre-filtered points...
npart: 1869391
Snapshot loaded.
Building tree...
 Done.
Making projection...
Projection complete.
Making projection...
Projection complete.


In [9]:
np.save("../data/vmap1.npy",vmap1)
np.save("../data/vmap2.npy",vmap2)

In [6]:
rbins = np.linspace(0,100,101)
dr = rbins[1]-rbins[0]

radii = np.linalg.norm(pos,axis=1)
dens = []
for ri in rbins:
    rmask = (radii > ri) & (radii < ri+dr)
    menc = np.sum(mass[rmask])*1e10*u.Msun
    vol = 4/3.*np.pi*((ri+dr)**3-ri**3)*u.kpc**3
    dens.append((menc/vol).to(u.Msun/u.kpc**3).value)
dens = np.array(dens)

In [11]:
np.save("../data/radial_density.npy",[rbins,dens])